# Using the generated SDK and data models

## Preparation

The main entry point for the SDK is the `PowerOpsClient`

To get started, we assume you have installed the `cognite-power-ops` SDK and that all configurations have been setup in Cognite Data Fusion (CDF).

Furthermore, it is assumed that you have setup one or two `toml` files with the credentials for connecting to Cognite Data Fusion and settings for this SDK.


### Interactive Login
This only requires one `toml` file looking like this

`settings.toml`

```toml
[cognite]
  login_flow = "interactive"
  project = "<cdf-project>"
  tenant_id = "<tenant-id>"
  cdf_cluster = "<cdf-cluster>"
  client_id = "<client-id>"

[powerops]
  read_dataset = "uc:000:powerops"
  write_dataset = "uc:000:powerops"
  monitor_dataset = "uc:po:monitoring"

```

### Client Credentials Login

For this case the you can use two toml files to separate the secrets from the regular settings.


`settings.toml`

```toml
[cognite]
  login_flow = "client_credentials"
  project = "<cdf-project>"
  tenant_id = "<tenant-id>"
  cdf_cluster = "<cdf-cluster>"
  client_id = "<client-id>"

[powerops]
  read_dataset = "uc:000:powerops"
  write_dataset = "uc:000:powerops"
  monitor_dataset = "uc:po:monitoring"
```
and the `.secrets.toml`

```toml
[cognite]
  client_secret = "<client-secret>"

```

Values in `.secrets.toml` will overwrite those in `settings.toml`




In [2]:
# You can control which setting files are loaded through the environmental variable below.
# In this case, the setting files are located two levels above, in the root of the repository.
import os
from cognite.powerops._version import __version__
from cognite.powerops import PowerOpsClient


os.environ["SETTINGS_FILES"] = "../../settings.toml;../../.secrets.toml"

powerops = PowerOpsClient.from_settings()

WRITE_DATA_SET = powerops.datasets.write_dataset_id

print(__version__)  # Print the version of the package

The generated modules `cog_shop1`, `assets`, `afrr_bid`, and `day_ahead_bid` are deprecated and will be removed.
`shop`/SHOPRunAPI is deprecated and will be removed.
`workflow`/DayaheadTriggerAPI is deprecated and will be removed.
0.98.4


In [2]:
# powerops.cogshop.trigger_shop_case("case_test_with_commands_3") # this words woo


## Setting up a ShopCase instance that can be triggered:
1. Upload a (set of) file(s) for a shop case
2. Set up a ShopCase, connecting them with a scenario
3. Write the ShopCase instance
4. Trigger a shop execution of that shop case
5. Retrieve a shop result, using the generated SDK


We will demonstrate different ways to generate ShopCases, and then how the results can be inspected

### Helpers for all types of case generation

In [13]:
from pathlib import Path
import datetime
from cognite.powerops.client._generated.v1.data_classes import (
    # These data classes are used to send data to CDF
    # There are non-write versions of these classes as well and they are used to read data from CDF
    ShopCaseWrite,  # This model contains everything needed to execute a SHOP run
    ShopFileWrite,  # A container that holds a reference to a file in CDF
    ShopModelWrite,  # A static model in a file reference + references to time series data
    ShopScenarioWrite,  # A way to modify run configurations for a given model
)


# Directory where the example case files are located
EXAMPLE_CASES_ROOT = Path("example_case_files")


# helper method for all case generations
def upload_file(
    file_name: str,
    external_id: str | None = None,
    mime_type: str = "application/yaml",
) -> str:
    """For simplicity we we will use the file name as the external id,
    but it can be can use any unique string as the external id."""
    external_id = external_id or file_name
    file_path = (EXAMPLE_CASES_ROOT / file_name).resolve()
    file = powerops.cdf.files.upload(
        path=str(file_path),
        external_id=external_id,
        name=file_name,
        data_set_id=WRITE_DATA_SET,
        mime_type=mime_type,
        # Overwrite the file at the given external is if it already exists
        # This will also overwrite potentially existing metadata
        overwrite=True,
    )
    return file.external_id


def upload_shop_case(shop_case: ShopCaseWrite):
    powerops.v1.upsert(shop_case)


def trigger_shop_case(case_external_id: str):
    powerops.cogshop.trigger_shop_case(case_external_id)


def upload_and_trigger(shop_case: ShopCaseWrite):
    upload_shop_case(shop_case)
    trigger_shop_case(shop_case.external_id)


# TODO
def get_result_instance(shop_case_external_id: str): ...

### Example A: Complete case file

In this case, the `ShopScenario` as and its `ShopModel` are mostly superfluous. 
However, they are still added as nearly empty objects in order to set the SHOP version 


In [39]:
def create_shop_case_a() -> ShopCaseWrite:
    # NB! this files does not have any cut groups, shop result is reasonable
    file_reference = upload_file(file_name="a_case_with_commands.yaml")

    shop_case_write = ShopCaseWrite(
        externalId="example_case_a",  # unique identifier for the case, used to trigger SHOP execution
        # The time range SHOP is optimized over
        startTime=datetime.datetime(2023, 9, 14, 22),
        endTime=datetime.datetime(2023, 9, 24, 22),
        # The scenario is used to modify the run configuration of the model
        scenario=ShopScenarioWrite(
            name="dummy_scenario",
            # The model is a static model in a file reference + references to time series data
            model=ShopModelWrite(
                name="dummy_model",
                shop_version="15.6.1.0",
            ),
        ),
        # The files that are used in the case
        shopFiles=[
            ShopFileWrite(
                name="a_case_with_commands",
                label="",
                fileReference=file_reference,  # external id of the file on CDF
                isAscii=False,
                order=1,
            ),
        ],
    )
    return shop_case_write

In [40]:
upload_and_trigger(create_shop_case_a())

### Example B: Almost complete case file

In this case, the `ShopScenario` as and its `ShopModel` are mostly superfluous. 
However, they are still added as nearly empty objects in order to set the SHOP version

A change here is that we extend the list in `ShopFile` and we need to specify the order that the files should be loaded into SHOP. 

Notice the label on the commands file. This is necessary as CogSHOP expects the commands to be labeled.  


In [41]:
def create_shop_case_b() -> ShopCaseWrite:
    # NB! this case does not have any cut groups, shop result is reasonable
    file_reference_case = upload_file(file_name="b_case_without_commands.yaml")
    file_reference_commands = upload_file(file_name="b_commands.yaml")

    shop_case_write = ShopCaseWrite(
        externalId="example_case_b",  # unique identifier for the case, used to trigger SHOP execution
        # The time range SHOP is optimized over
        startTime=datetime.datetime(2023, 9, 14, 22),
        endTime=datetime.datetime(2023, 9, 24, 22),
        # The scenario is used to modify the run configuration of the model
        scenario=ShopScenarioWrite(
            name="dummy_scenario",
            # The model is a static model in a file reference + references to time series data
            model=ShopModelWrite(
                name="dummy_model",
                shop_version="15.6.1.0",
            ),
        ),
        # The files that are used in the case
        shopFiles=[
            ShopFileWrite(
                name="b_case_without_commands",
                label="",
                fileReference=file_reference_case,  # external id of the file on CDF
                isAscii=False,
                order=1,
            ),
            ShopFileWrite(
                name="b_commands",
                label="commands",
                fileReference=file_reference_commands,  # external id of the file on CDF
                isAscii=False,
                order=2,
            ),
        ],
    )
    return shop_case_write

In [42]:
upload_and_trigger(create_shop_case_b())